# <center>**Trabajo Practico de Estructura de datos**
## **Solver sudokus:** 

### <font color=yellow>Decisiones de diseño:</font>
- Se decidió utilizar una lista con sub-listas para representar el archivo provisto por el usuario. De forma que cada línea del csv será una fila del tablero.
- Se agrego el modulo "Colorama" en primer lugar para darle color y formato al texto, o tablero.
- Luego de introducir el archivo, también se pide el valor de la dimensión para comprobar que el tablero sea simétrico
- Al terminar de leer el tablero y guardarlo en la estructura, se procederá a resolver este tablero para poder aprovechar mejor los recursos disponibles. Cuando se termina se escribe en un csv de salida predeterminado.
- Los archivos generados se guardan en una subcarpeta llamada "archivos".
- Para la medición de tiempos se creó una clase para generar tableros.
- Cuando se busca medir tiempos, para simular fallos en el backtracking, se implementó un método para mantener controlada la cantidad de intentos, de forma que cuando se verifica y no hay espacio libre, se controla la cantidad con el método y se devuelve False.
- El método _resolver, que implementa parte del backtracking, deberá recibir -1 como 3° parámetro para la medición de tiempos. Los últimos 2 parámetros tienen valores predeterminados, por que en algunos casos no es necesario utilizarlos.
- Se decidió que al presionar Ctrl+C para detener la ejecución del solver se guarde el tablero, el núm. de línea en el que termina en el archivo de entrada, para luego terminar de resolver el tablero actual si es posible y continuar con la resolución del resto de sudokus.
- Para la persistencia, se utilizó json.
- Se decidió agregar color a los mensajes de error, líneas para diferenciar sectores del tablero y núm. de línea para saber que sudoku es. En primer lugar se utilizo un modulo llamado "Colorama", pero luego para evitar uso de este usó códigos de secuencias o códigos de escape del estándar ANSI para cambiar los formatos de las salidas. Colorama se usará solo para dar "estilo".
- Se decidió por la interpretación del TP, que de una vez que el proceso actual es guardado se cierre la ventana. Ya que no parecería necesario mantenerla abierta debido a que la idea es terminar el proceso de ejecución.
- A algunos métodos se les agrego un guion bajo adelante para "simular" o seguir la convención de que es privado
---

### <font color=yellow>Descripción breve de cada archivo fuente y funciones:</font>
- ***Resolvedor.py***:
    - **clase Solucionador_sudokus**
        - suma_fallos_provocados: Mantendra cuenta y devolvera los fallos para controlarlos
        - _carga_archivo_y_resuelve: Maneja excepciones de archivo, lee el archivo, lo procesa y luego llama _resolver
        - _resolver: tiene opcion de simular fallos, implementa parte del backtracking para resolver. Cuando resuelve lo escribe en csv
        - _pintar_separador_horizontal: imprime linea horizontal, se utilizara varias veces
        - _mostrar_tablero_consola: imprime el tablero, separando por el modulo de la longitud de la submatriz, tanto filas como columnas
        - _buscar_espacio_libre: itera sobre el tablero buscando 0 y si encuentra devuelve una tupla de los valores
        - guardar_progreso_json: guarda el diccionario con datos relevantes. En caso de existencia del archivo, lo sobreescribe
        - _recuperar_progreso_con_json: itera y maneja los datos para resolver tablero y luego continuar la ejecucion
        - medir_tiempos_de_diferentes_dimensiones: para tableros de dimension: 9 y 16 saca un promedio del tiempo de 10 iteraciones. En dimension 25 solo muestra el tiempo de una solucion
- ***App.py***:
    - resolver_sudokus: Mostrara las opciones del programa y manejara la eleccion de estas
    - main: ejecuta el resolvedor
- ***Excepciones_solver.py***:
    - **clase Solver_sudoku_exception**
        - sera la super clase de excepciones propias
    - **clase Tipo_de_archivo_incorrecto_exception**
        - en caso de que el archivo no tenga extension .csv
    - **clase Dimension_incorrecta_exception**
        - constructor: tomara como parametro la dimension y manejara el mensaje a arrojar
        - __str__: utilizado para devolver los respectivos mensajes
    - **clase Opcion_incorrecta_exception**
        - __str__: mostrara el error correspondiente a opcion incorrecta
- Generador_Sudokus.py:
    - **clase Generador_Sudokus**
        - generar_sudokus_en_csv: recibe dimension y la cantidad de tableros a crear. Asigna la 3° parte del tablero con valores random, verificados.
        - _generar_tablero_en_matriz: genera una lista de listas como tablero
- ***Verificador_de_reglas***:
    - **clase Verificador_reglas**
        - verificar_posicion_libre: con la longitud de la submatriz divide fila y columna, pasando a entero para eliminar decimales y luego multiplica por esta long, buscando obtener el inicio de fila-columna para iterar. Con los valores recibidos de fila-columna itera sobre la matriz, fijando un valor a la vez. Devuelve falso si no es posible introducir el num
        - verificar_tamaño_del_tablero: recibe la dimension introducida por el usuario y la compara con la longitud de cada fila, para evitar que sea una matriz irregular. Corta en caso de error
        - verificar_valores_predeterminados_correctos: verifica que los valores predeterminados no sean menor que 0 y mayor que el valor de la dimension. Devuelve True en un caso correcto
---
#### Se agrega parte del codigo a ejecutar:
- Debido a que falta gran parte del codigo, quiza no se resuelvan algunas cosas como se deberia. Mas que anda excepciones

In [2]:
import csv;from math import sqrt as raiz_cuadrada
formato_predeterminado = '\033[0;37m'
def _pintar_separador_horizontal():
    print('\033[0;31m' + " ---------------------------"+formato_predeterminado)


def _mostrar_tablero_consola(tablero, dimension):
    formato_separador_columna = '\033[0;31m'
    long_submatriz = int(raiz_cuadrada(dimension))
    _pintar_separador_horizontal()
    for i in range(dimension):
        if i % long_submatriz == 0 and i != 0:  # 3/3 6/3 9*9
            _pintar_separador_horizontal()

        for j in range(dimension):
            if j % long_submatriz == 0:
                print(formato_separador_columna + " | "+formato_predeterminado, end="")

            if j == dimension - 1:
                print(str(tablero[i][j]) + " ", end="")
                print(formato_separador_columna + "|"+formato_predeterminado)
            else:
                print(str(tablero[i][j]) + " ", end="")

    _pintar_separador_horizontal()
    print('\033[0;32m' + "___________________________________\n" +formato_predeterminado)
def verificar_posicion_libre(tablero, valor, posicion, dimension):
#     self.verificar_tamaño_del_tablero(tablero,dimension)
    posicion_disponible=True
    columna_a_verificar=posicion[1]
    longitud_submatriz=int(raiz_cuadrada(dimension))#el metodo devuelve float
    inicio_columna = (int(posicion[1] / longitud_submatriz)) * longitud_submatriz  #  ->pos 14 / 4 int le saca decimal
    inicio_fila = (int(posicion[0] / longitud_submatriz)) * longitud_submatriz
    for fila in range(inicio_fila,inicio_fila + longitud_submatriz):
        for columna in range(inicio_columna, inicio_columna + longitud_submatriz):
            if tablero[fila][columna] == valor:
                posicion_disponible = False
                return posicion_disponible

    fila_a_verificar=tablero[posicion[0]]
    if valor in fila_a_verificar:
        posicion_disponible = False
        return posicion_disponible

    for fila in range(dimension):
        if tablero[fila][columna_a_verificar] == valor:
            posicion_disponible = False
            return posicion_disponible
    return posicion_disponible

def _carga_archivo_y_resuelve(archivo_entrada,dimension,num_linea_recuperacion=0):
    '''recibe un archivo csv con los numeros separados por coma y otro parametro para
        comprobar que la dimension de todos los tableros sea la correcta, debe ser un entero'''
    # try:
    formato_error='\033[4;31m'
#         if not (0 < dimension and type(dimension) is  int and raiz_cuadrada(dimension) / int(raiz_cuadrada(dimension))==1):
#             raise Dimension_incorrecta_exception(dimension)
#         if not archivo_entrada.__contains__(".csv"):
#             raise Tipo_de_archivo_incorrecto_exception(archivo_entrada)
    archivo = csv.reader(open(archivo_entrada, "r"), delimiter=",")
    fila_temporal, tablero = list(), list()
    num_linea = 0
    for linea in archivo:
        if num_linea_recuperacion is not 0:#va a saltear las lineas necesarias
            num_linea_recuperacion-=1
            continue
        num_linea += 1
        for numero in linea:
            fila_temporal.append(int(numero))
        tablero.append(fila_temporal)
        #clear?
        fila_temporal = list()
        if num_linea % dimension == 0:
            print('\033[1;33m' + f"Sudoku N°:{int(archivo.line_num / dimension)}")
            print('\033[1;33m'+ f"Linea de archivo N°:{archivo.line_num}")
                    #termina de cargar el tablero, y lo resuelve
#                     if Verificador_reglas().verificar_tamaño_del_tablero(tablero, dimension) \
#                             and Verificador_reglas().verificar_valores_predeterminados_correctos(tablero,dimension):
            _resolver(tablero,dimension,num_linea,archivo_entrada)
#                     elif not Verificador_reglas().verificar_tamaño_del_tablero(tablero, dimension):
#                         # raise Tablero_irregular_exception
#                         print(formato_error+"El tablero debe tener la misma cantidad de filas y columnas, del mismo tamaño")
#                     else:
#                         print(formato_error+f"El tablero solo puede contener valores entre 0 y {dimension}")
            tablero.clear()
        # print('\033[1;33m'+ str(archivo.line_num))
#         except Tipo_de_archivo_incorrecto_exception as e:
#             print(e)
#             esperar(5)
#         except Dimension_incorrecta_exception as e:
#             print(e)
#             esperar(5)
#         except Termino_ejecucion_exception:
#             raise KeyboardInterrupt
def _resolver(tablero,dimension,num_linea=0,archivo_entrada=None):#no se  puede poner parametro por "defecto"
    #agrego num_linea para poder guardar el num_linea por el cual se quedo la lectura el archivo de entrada
    # try:
    espacio_libre = _buscar_espacio_libre(tablero,dimension)
    if not espacio_libre:
#             if num_linea == -1 :
#                 if self.suma_fallos_provocados()<=10:
#                     # print("'fallo' provocado")#entra las 10 veces...
#                     return False
#             else:
        temporal = csv.writer(open("salida_tableros.csv", "a", newline=""))
        for j in tablero:
            temporal.writerow(j)
        _mostrar_tablero_consola(tablero, dimension)
        return True
    else:

        fila, columna = espacio_libre

        for i in range(1,dimension+1):
            if verificar_posicion_libre(tablero, i, (fila, columna),dimension):
                tablero[fila][columna] = i

                if _resolver(tablero,dimension,num_linea,archivo_entrada):
                    return True

                tablero[fila][columna] = 0

        return False
#     except KeyboardInterrupt:
#         path_archivo_serializacion = input("Introduzca el path donde querra guardar la resolucion parcial: ")
#         #a dic de datos se le tiene que pasar: tablero actual, num_linea, dimension, y archivo de entrada
#         dic_datos={"tablero":tablero,
#                     "num_linea":num_linea,
#                     "dimension":dimension,
#                     "ruta_entrada":archivo_entrada}
#         # print(tablero,num_linea,dimension,archivo_entrada,path_archivo_serializacion)
#         self.guardar_progreso_json(dic_datos,path_archivo_serializacion)
#         sys.exit()
def _buscar_espacio_libre(tablero,dimension):
    for i in range(dimension):
        for j in range(dimension):
            if tablero[i][j] == 0:
                return (i, j)
_carga_archivo_y_resuelve("tableros.csv",9)

Sudoku N°:1
Linea de archivo N°:9
 ---------------------------
 | 7 8 5  | 4 3 9  | 12 2 6 |
 | 6 1 2  | 8 7 5  | 3 4 9 |
 | 4 9 3  | 6 2 1  | 5 7 8 |
 ---------------------------
 | 8 5 7  | 9 4 3  | 2 6 1 |
 | 2 6 1  | 7 5 8  | 9 3 4 |
 | 9 3 4  | 1 6 2  | 7 8 5 |
 ---------------------------
 | 5 7 8  | 3 9 4  | 6 1 2 |
 | 1 2 6  | 5 8 7  | 4 9 3 |
 | 3 4 9  | 2 1 6  | 8 5 7 |
 ---------------------------
___________________________________

Sudoku N°:2
Linea de archivo N°:18
Sudoku N°:3
Linea de archivo N°:27
 ---------------------------
 | 1 7 3  | 4 8 6  | 5 9 2 |
 | 4 2 5  | 9 1 7  | 6 8 3 |
 | 9 6 8  | 3 2 5  | 4 1 7 |
 ---------------------------
 | 6 8 4  | 1 7 2  | 3 5 9 |
 | 5 9 7  | 6 3 4  | 8 2 1 |
 | 3 1 2  | 5 9 8  | 7 6 4 |
 ---------------------------
 | 2 4 9  | 8 5 3  | 1 7 6 |
 | 7 5 6  | 2 4 1  | 9 3 8 |
 | 8 3 1  | 7 6 9  | 2 4 5 |
 ---------------------------
___________________________________

Sudoku N°:4
Linea de archivo N°:36
 ---------------------------
 | 

---
### <font color=yellow>Comparacion de medicion con analisis a mano:</font>
Considerando emular o generar fallos para encontrar todas las soluciones posibles se va a dar el hecho de probar n veces en cada casillero por cada fallo que llegue al inicio, la complejidad seria O(n^n) en el peor de los casos. En un tablero con algunos valores por defecto el consumo de recursos seria menor, ya que habria menos posibilidades de resolucion.

---
### <font color=yellow>Conclusión:</font>
Resulto compleja la recuperacion de una resolucion parcial, debido a que en la implementacion utilizada, no se pudo encontrar forma de simular el proceso o stack de recursividad perdido al guardar los datos. En algunos casos, muy pocos, funcionara a la perfeccion la recuperacion del tablero parcialmente resuelto, pero el resto de los tableros que siguen no tendran problema.
Estuvo bueno tener la oportunidad de comparar una medicion de tiempos real, con respecto a un analisis de la complejidad del algoritmo.
Hubiese estado bueno hacer que el programa se detuviera con otra combinacion de teclas, para poder ver mejor los eventos en python.